In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
import os
import numpy as np
import tensorflow as tf
from keras.models import model_from_json
import cv2
import predict
import matplotlib.pyplot as plt
%matplotlib inline
kernel = np.ones((2,2),np.uint8)

Using TensorFlow backend.


[[0.01300771 0.03071591 0.8094308  0.04651451 0.0118816  0.0043277
  0.00862417 0.03615722 0.03261816 0.00672223]]


In [47]:

def predict1(img):
    img = cv2.erode(img,  kernel, iterations = 1)
#     plt.imshow(img, cmap = "gray")

    img = cv2.resize(img, (28, 28))
    img = 255-img
#     img =  img[:, 10:]
#     tmp_arr = np.zeros((28, 10))
#     print(tmp_arr.shape)
#     img = np.concatenate((img, tmp_arr), axis  =1)
    
#     print(img)
    for i in range(28):
        for j in range(28):
            if(img[i][j]>20):
                img[i][j] = 250

#     img = cv2.dilate(img,  kernel, iterations = 1)
#     print(img)
#     plt.imshow(img)
#     img = 255-img
#     plt.imshow(img, cmap = "gray")
    img = np.reshape(img, (1,28,28,1))
    
    model1 = Sequential()
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")


    model_new = Sequential()
    weights = loaded_model.get_weights()
   # model_new.set_weights(weights)
    #model_new.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    pred_val = loaded_model.predict(img, )
    pred_val = pred_val[0]
#     print(pred_val)
    res = 0
    confidence = 0
    for i in range(10):
        if confidence < pred_val[i]:
            confidence = pred_val[i]
            res = i
    return [res, confidence]
# predict1("images/4.jpg")


In [48]:
def contains_two(v_count, min_indx):
#     print(len(v_count), min_indx)
    peak_left = 0
    peak_right = 0
    for i in range(min_indx):
        if peak_left < v_count[i]:
            peak_left = v_count[i]
    for i in range(min_indx, len(v_count)):
        if peak_right < v_count[i]:
            peak_right = v_count[i]
#     print("peaks: ", peak_left, peak_right)
#     print("min_val:" , v_count[min_indx])
    if(min(peak_left, peak_right) - v_count[min_indx])>10:
        return True
    if v_count[min_indx]<3:
        return True
    return False

In [49]:
def crop_image(img, v_count):
    left_index = 0
    count = 0
    new_v_vount = []
    while(count < 10):
        count += v_count[left_index]
        left_index+=1
    right_index = len(v_count)-1
    count = 0
    while(count<10):
        count += v_count[right_index]
        right_index-=1
    img = img[:, max(left_index-10, 0) : min(right_index+10, len(v_count))]
    for i in range(max(left_index-10, 0), min(right_index+10, len(v_count))):
        new_v_vount.append(v_count[i])
    return img, new_v_vount
    

In [50]:
def get_v_count(img):
    x = img.shape[1]
    y = img.shape[0]
    v_count = np.zeros((x,))
    for i in range(x):
        for j in range(y):
            if(img[j][i] <230):
                v_count[i]+= 1
    return v_count

In [51]:
def padd_image(img):
    x, y = img.shape
    r = 0
    if x>y:
        r = int((x-y)/2)
    tmp = np.zeros((x, r))
    tmp = 255-tmp
#     print("tmp: ", tmp.shape)
    img = np.concatenate((tmp, img), axis = 1)
    img = np.concatenate((img, tmp),  axis = 1)
#     print(img.shape)
    return img

In [52]:

def get_prefix_sum(v_count):
    x = len(v_count)
    prefix_sum = np.zeros((x,))
    prefix_sum[0] = v_count[0]
    for i in range(1, x):
        prefix_sum[i] = v_count[i] + prefix_sum[i-1]
        
    return prefix_sum

In [53]:
def find_threshold(filepath):

    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plt.imshow(img)
    _,img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    v_count = get_v_count(img)
    img, v_count = crop_image(img, v_count)
    x = len(v_count)
    plt.imshow(img, cmap = "gray")

    prefix_sum = get_prefix_sum(v_count)
    l = int(prefix_sum[x-1]*0.2)
    r = int(prefix_sum[x-1]*0.8)
    i = 0
    while( l > prefix_sum[i]):
        i = i+1
    l_r = i
    while( r > prefix_sum[i]):
        i = i+1
    r_r = i
    min_val = v_count[l_r];
    min_indx = l_r
#     print(v_count)
#     print("l_r", l_r, "r_r", r_r)

    for i in range(l_r-1, r_r+1):
        if(min_val > v_count[i]):
            min_val = v_count[i]
            min_indx = i
#             print("vvt ", min_indx)
#     print(min_indx, len(v_count))
    res = None
    if(not contains_two(v_count, min_indx)):
        img = padd_image(img)
        res = predict1(img)
        print(res, "nj")
        return
    msb = img[:, :min_indx+3]
    lsb = img[: , min_indx:]
    msb = padd_image(msb)
    print(predict1(msb))
    plt.imshow(msb, cmap = "gray")
# find_threshold("images/13.jpg")

In [54]:
def predict_double(img, min_indx, l_r, r_r):
    prediction = []
#     for i in range(l_r, r_r):
#         if i%5==0:
    msb = img[:, :min_indx+3]
    lsb = img[: , min_indx:]
    msb = padd_image(msb)
    lsb = padd_image(lsb)
    msb = predict1(msb)
    lsb = predict1(lsb)
    msb[0] = msb[0]*10 + lsb[0]
    msb[1] = msb[1]+lsb[1]
    msb[1]/=2
    prediction.append(msb)
#     print('v')
    return prediction

In [55]:
def get_text(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     plt.imshow(img)
    _,img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    v_count = get_v_count(img)
    img, v_count = crop_image(img, v_count)
    x = len(v_count)
#     plt.imshow(img, cmap = "gray")

    prefix_sum = get_prefix_sum(v_count)
    l = int(prefix_sum[x-1]*0.2)
    r = int(prefix_sum[x-1]*0.8)
    i = 0
    while( l > prefix_sum[i]):
        i = i+1
    l_r = i
    while( r > prefix_sum[i]):
        i = i+1
    r_r = i
    prediction = []
#     print(prediction)
    min_val = v_count[l_r];
    min_indx = l_r
#     print(v_count)
#     print("l_r", l_r, "r_r", r_r)
    
    for i in range(l_r-1, r_r+1):
        if(min_val > v_count[i]):
            min_val = v_count[i]
            min_indx = i
    if not contains_two(v_count, min_indx):
        tmp_img = padd_image(img)
        prediction.append(predict1(tmp_img))
    pred_tmp = predict_double(img, min_indx, l_r, r_r)
    for val in pred_tmp:
        prediction.append(val)
    prediction = sorted(prediction, key = lambda x:x[1])
    
    if prediction[-1][1] < 0.95:
        print(prediction[-1][0])
        return None
    return  prediction[-1]
#     print(l_r, r_r)
        
print(get_text("images/3A.jpg"))

[3, 0.9728468]
